In [1]:
import pandas as pd
from datetime import datetime
import glob
import os
import math
from IPython.display import display
from IPython.display import display, HTML


# To be able to read XLS files
# pip install xlrd

In [2]:
path_qgis = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+Combining ALL DATA with Agro\PhenoCrop 2023'

In [3]:
path_agro = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+Combining ALL DATA with Agro\PhenoCrop 2023\AGRO DATA 2023'

In [4]:
all_files_qgis = glob.glob(path_qgis + "/*.xlsx")
# all_files_qgis

In [5]:
# Importing all the datasets except those from Mavic3RGB images since we need the Multispectral data only

list_df_names = []
for filepath in all_files_qgis:
    if not("M3RGB") in filepath:
        file_name=os.path.basename(filepath[:-5]).replace('-','_')
        list_df_names.append(file_name)
        
        # print(filepath,os.path.basename(filepath[:-5]))
        locals()[file_name] = pd.read_excel(filepath, index_col=None, header=0)
        print(file_name, '=====', locals()[file_name].shape)

phenocrop_2023_P4M_20m_20230623 ===== (760, 19)
phenocrop_2023_P4M_20m_20230628 ===== (760, 19)
phenocrop_2023_P4M_20m_20230708 ===== (760, 28)
phenocrop_2023_P4M_20m_20230714 ===== (760, 16)
phenocrop_2023_P4M_20m_20230719 ===== (760, 19)
phenocrop_2023_P4M_20m_20230727 ===== (760, 19)
phenocrop_M3MS_20m_20230614 ===== (760, 16)
Phenocrop_M3MS_20m_20230622 ===== (760, 16)
phenocrop_M3MS_20m_20230628 ===== (760, 16)
phenocrop_M3MS_20m_20230708 ===== (760, 16)
phenocrop_M3MS_20m_20230714 ===== (760, 16)


In [6]:
# Since one dataset has more than 19 columns,
# and after checking them manually it was found that the ones after the 19th columns are not relevant,
# keeping only the first 19 columns here.

for dfs in list_df_names:
    # print(dfs, locals()[dfs].shape)
    locals()[dfs] = locals()[dfs].iloc[:,:19]
    print(dfs, locals()[dfs].shape)

phenocrop_2023_P4M_20m_20230623 (760, 19)
phenocrop_2023_P4M_20m_20230628 (760, 19)
phenocrop_2023_P4M_20m_20230708 (760, 19)
phenocrop_2023_P4M_20m_20230714 (760, 16)
phenocrop_2023_P4M_20m_20230719 (760, 19)
phenocrop_2023_P4M_20m_20230727 (760, 19)
phenocrop_M3MS_20m_20230614 (760, 16)
Phenocrop_M3MS_20m_20230622 (760, 16)
phenocrop_M3MS_20m_20230628 (760, 16)
phenocrop_M3MS_20m_20230708 (760, 16)
phenocrop_M3MS_20m_20230714 (760, 16)


# One dataset has blue values missing. Will have to fix that processing in QGIS later. Right now moving forward with the missing values.

# Adding date column

In [7]:
# Adding respective date column in each dataset
for dfs in list_df_names:

    date_from_dfs_name = dfs[-8:]
    # Converting date str to datetime object
    date_obj = datetime.strptime(date_from_dfs_name, '%Y%m%d')
    # Converting datetime obj to formatted date str
    date_str = date_obj.strftime('%m-%d-%Y')

    # Adding a column with formatted date in the dataset
    locals()[dfs]["Date"]= date_str
    print(dfs, locals()[dfs].shape)

phenocrop_2023_P4M_20m_20230623 (760, 20)
phenocrop_2023_P4M_20m_20230628 (760, 20)
phenocrop_2023_P4M_20m_20230708 (760, 20)
phenocrop_2023_P4M_20m_20230714 (760, 17)
phenocrop_2023_P4M_20m_20230719 (760, 20)
phenocrop_2023_P4M_20m_20230727 (760, 20)
phenocrop_M3MS_20m_20230614 (760, 17)
Phenocrop_M3MS_20m_20230622 (760, 17)
phenocrop_M3MS_20m_20230628 (760, 17)
phenocrop_M3MS_20m_20230708 (760, 17)
phenocrop_M3MS_20m_20230714 (760, 17)


# Importing AGRO DATA

In [33]:
all_files_agro = glob.glob(path_agro + "/*.xls")
all_files_agro

['C:\\Users\\muij\\OneDrive - Norwegian University of Life Sciences\\Documents\\+Combining ALL DATA with Agro\\PhenoCrop 2023\\AGRO DATA 2023\\23TVollebekk-avling data fra JAD_231106.xls']

In [34]:
# Listing all sheets in the xls file
sheets_agro = []

for filepath in all_files_agro:
    file_name=os.path.basename(filepath).replace('-','_')
    agro_data_obj = pd.ExcelFile(filepath)
    sheets_in_xls = pd.ExcelFile(filepath).sheet_names

    # Importing data from the sheets to respective sheeet name+'df_' prefix
    for sheet in sheets_in_xls:
        sheet_df = 'df_'+sheet
        locals()[sheet_df] = agro_data_obj.parse(sheet)
        sheets_agro.append(sheet_df)
        print(locals()[sheet_df].shape)

(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)


In [35]:
# Fixing the column names in the agro data

for sheet_df in sheets_agro:
    df_temp = locals()[sheet_df].copy()
    cols_row1 = df_temp.columns[:].tolist()
    cols_row2 = df_temp.iloc[0,:].tolist()
    
    # Eliminating Nan values in the lists
    cols_row1 = ['' if pd.isna(x) else x for x in cols_row1]
    cols_row2 = ['' if pd.isna(x) else x for x in cols_row2]
    
    # Eliminating unnamed entries in the first column
    cols_row1 = ['' if 'Unnamed' in x else x for x in cols_row1]
    # Lising the corrected columns
    cols_corrected = list(map(''.join, zip(cols_row1, cols_row2)))

    # Correcting a mistake in column headings where the header after Plot is sometimes named entry and others bloc and also empty in one case
    cols_corrected[3] = 'entry/bloc'

    # Renaming the PLOT column heading to ID
    cols_corrected[2] = 'id'
    
    # Assigning new corrected columns to the sheet
    df_temp.columns = cols_corrected

    #Assigning the processed data to the original sheet df name
    locals()[sheet_df] = df_temp.copy()
    print(locals()[sheet_df].shape)

(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)
(52, 17)


In [36]:
# Droppping the unnecessary rows originating from the column headings in the xls file
for sheet_df in sheets_agro:
    df_temp2 = locals()[sheet_df].copy()

    # Finding the row with all Nan entries since the rows before that one are the column headers
    for row in range(df_temp2.shape[0]):
        # print(row)
        # the following code can cause problems if here are multiple empty rows in a dataset
        if df_temp2.iloc[row,:].isnull().all():
            cutoff_row = row
    
    # Dropping the false heading rows based on the cutoff_row found out in the previous step 
    df_temp2 = df_temp2.drop(df_temp2.index[range(cutoff_row+1)]).reset_index(drop=True)

    #Assigning the processed data to the original sheet df name
    locals()[sheet_df] = df_temp2.copy()
    print(locals()[sheet_df].shape)

(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)


In [37]:
# # Printing all the headers/column headings for all sheets
# for sheets in sheets_agro:
#     print(locals()[sheets].shape[1])
#     display(locals()[sheets].head(0))

In [38]:
for sheets in sheets_agro:
    print(locals()[sheets].shape)

(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)
(50, 17)


# Conct all sheets

In [39]:
full_agro = locals()[sheets_agro[0]].copy()

for sheet_df in sheets_agro[1:]:
    full_agro = pd.concat([full_agro, locals()[sheet_df]], axis=0)
    
display(full_agro)


,r_expt,r_location,id,entry/bloc,,name,pedigree,Legde,Pl.høyde,Etter-Ren.,Akssk.Juni,Gulm.Aug.,Rå rute-avling,Tørr rute-avling,Vanntørr,%Vann,Avlingkg/daa,tørr rute-avling
0,23T1A 4,Vollebekk,107,8.0,NaN,Festus,Demonstrant/SW51114,0,66,25,27,22,3.87,3171,11.7,27.64876,439.214588,NaN
1,23T1A 4,Vollebekk,108,24.0,NaN,GN22529,Russland4/GN05567,0,67,20,25,21,3.07,2596,11.7,25.33329,359.571451,NaN
2,23T1A 4,Vollebekk,109,18.0,NaN,GN18715,GN07574/GN05567,0,69,20,25,22,3.97,3248,11.7,27.758589,449.879843,NaN
3,23T1A 4,Vollebekk,110,13.0,NaN,Leijona,NaN,0,63,20,28,22,3.76,3141,11.7,26.236622,435.059294,NaN
4,23T1A 4,Vollebekk,111,1.0,NaN,Zebra,Ralle/Dragon,0,67,20,26,21,3.67,3066,11.7,26.232207,424.671059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,23T12B 4,Vollebekk,885,2,NaN,GN23669,GN06578/GN08530,0,72,5,29,23,5.16,4169,12.3,29.143159,573.523608,NaN
46,23T12B 4,Vollebekk,886,2,NaN,GN23679,Amulett/GN08581,0,72,2,27,21,4.47,3741,12.3,26.602752,514.644235,NaN
47,23T12B 4,Vollebekk,887,2,NaN,GN23676,SW21307/Mirakel,0,72,5,27,26,5.78,4658,12.3,29.324118,640.794667,NaN
48,23T12B 4,Vollebekk,888,2,NaN,GN23665,GN09543/GN07501,0,81,5,28,26,4.9,4038,12.3,27.728041,555.502118,NaN


In [40]:

# display(HTML(pd.concat([df_23T1A, df_23T2A], axis=0).to_html()))
# pd.concat([df_23T1A, df_23T2A], axis=0).shape

# Merge the Agro data with the indices data

In [41]:
list_df_names

['phenocrop_2023_P4M_20m_20230623',
 'phenocrop_2023_P4M_20m_20230628',
 'phenocrop_2023_P4M_20m_20230708',
 'phenocrop_2023_P4M_20m_20230714',
 'phenocrop_2023_P4M_20m_20230719',
 'phenocrop_2023_P4M_20m_20230727',
 'phenocrop_M3MS_20m_20230614',
 'Phenocrop_M3MS_20m_20230622',
 'phenocrop_M3MS_20m_20230628',
 'phenocrop_M3MS_20m_20230708',
 'phenocrop_M3MS_20m_20230714']

In [42]:
full_agro

,r_expt,r_location,id,entry/bloc,,name,pedigree,Legde,Pl.høyde,Etter-Ren.,Akssk.Juni,Gulm.Aug.,Rå rute-avling,Tørr rute-avling,Vanntørr,%Vann,Avlingkg/daa,tørr rute-avling
0,23T1A 4,Vollebekk,107,8.0,NaN,Festus,Demonstrant/SW51114,0,66,25,27,22,3.87,3171,11.7,27.64876,439.214588,NaN
1,23T1A 4,Vollebekk,108,24.0,NaN,GN22529,Russland4/GN05567,0,67,20,25,21,3.07,2596,11.7,25.33329,359.571451,NaN
2,23T1A 4,Vollebekk,109,18.0,NaN,GN18715,GN07574/GN05567,0,69,20,25,22,3.97,3248,11.7,27.758589,449.879843,NaN
3,23T1A 4,Vollebekk,110,13.0,NaN,Leijona,NaN,0,63,20,28,22,3.76,3141,11.7,26.236622,435.059294,NaN
4,23T1A 4,Vollebekk,111,1.0,NaN,Zebra,Ralle/Dragon,0,67,20,26,21,3.67,3066,11.7,26.232207,424.671059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,23T12B 4,Vollebekk,885,2,NaN,GN23669,GN06578/GN08530,0,72,5,29,23,5.16,4169,12.3,29.143159,573.523608,NaN
46,23T12B 4,Vollebekk,886,2,NaN,GN23679,Amulett/GN08581,0,72,2,27,21,4.47,3741,12.3,26.602752,514.644235,NaN
47,23T12B 4,Vollebekk,887,2,NaN,GN23676,SW21307/Mirakel,0,72,5,27,26,5.78,4658,12.3,29.324118,640.794667,NaN
48,23T12B 4,Vollebekk,888,2,NaN,GN23665,GN09543/GN07501,0,81,5,28,26,4.9,4038,12.3,27.728041,555.502118,NaN


In [60]:
list_df_agro = []

for dfs in list_df_names:
    dfs_temp = dfs+'_agro'
    locals()[dfs_temp] = pd.merge(full_agro, locals()[dfs], on="id")
    list_df_agro.append(dfs_temp)

In [61]:
list_df_agro

['phenocrop_2023_P4M_20m_20230623_agro',
 'phenocrop_2023_P4M_20m_20230628_agro',
 'phenocrop_2023_P4M_20m_20230708_agro',
 'phenocrop_2023_P4M_20m_20230714_agro',
 'phenocrop_2023_P4M_20m_20230719_agro',
 'phenocrop_2023_P4M_20m_20230727_agro',
 'phenocrop_M3MS_20m_20230614_agro',
 'Phenocrop_M3MS_20m_20230622_agro',
 'phenocrop_M3MS_20m_20230628_agro',
 'phenocrop_M3MS_20m_20230708_agro',
 'phenocrop_M3MS_20m_20230714_agro']

In [64]:
phenocrop_2023_P4M_20m_20230623_agro

display(HTML(phenocrop_2023_P4M_20m_20230623_agro.to_html()))


,r_expt,r_location,id,entry/bloc,,name,pedigree,Legde,Pl.høyde,Etter-Ren.,Akssk.Juni,Gulm.Aug.,Rå rute-avling,Tørr rute-avling,Vanntørr,%Vann,Avlingkg/daa,tørr rute-avling,ndvi_mean,ndvi_median,ndvi_stdev,blue_mean,blue_median,blue_stdev,green_mean,green_median,green_stdev,nir_mean,nir_median,nir_stdev,red_edge_mean,red_edge_median,red_edge_stdev,red_mean,red_median,red_stdev,Date
0,23T1A 4,Vollebekk,107,8.0,NaN,Festus,Demonstrant/SW51114,0,66,25,27,22,3.87,3171,11.7,27.64876,439.214588,NaN,0.744566,0.760509,0.072517,0.027839,0.027391,0.005295,0.046787,0.045989,0.009926,0.257103,0.253312,0.067270,0.143535,0.141264,0.034494,0.035824,0.035081,0.007545,06-23-2023
1,23T1A 4,Vollebekk,108,24.0,NaN,GN22529,Russland4/GN05567,0,67,20,25,21,3.07,2596,11.7,25.33329,359.571451,NaN,0.813557,0.829784,0.060461,0.023538,0.023264,0.004226,0.040980,0.040417,0.008518,0.283151,0.287463,0.069643,0.145958,0.147083,0.032814,0.027515,0.026834,0.005707,06-23-2023
2,23T1A 4,Vollebekk,109,18.0,NaN,GN18715,GN07574/GN05567,0,69,20,25,22,3.97,3248,11.7,27.758589,449.879843,NaN,0.778521,0.791565,0.057855,0.025517,0.025070,0.004869,0.044059,0.043256,0.009565,0.263364,0.261192,0.065646,0.143123,0.141644,0.033515,0.031416,0.030750,0.006323,06-23-2023
3,23T1A 4,Vollebekk,110,13.0,NaN,Leijona,NaN,0,63,20,28,22,3.76,3141,11.7,26.236622,435.059294,NaN,0.814132,0.822713,0.045319,0.025654,0.025362,0.004711,0.043325,0.042854,0.008849,0.293979,0.297382,0.066336,0.151097,0.152030,0.032390,0.029030,0.028630,0.005299,06-23-2023
4,23T1A 4,Vollebekk,111,1.0,NaN,Zebra,Ralle/Dragon,0,67,20,26,21,3.67,3066,11.7,26.232207,424.671059,NaN,0.793360,0.802693,0.051203,0.025695,0.025300,0.005038,0.043953,0.043293,0.009503,0.275623,0.273239,0.067332,0.146759,0.145755,0.033703,0.030572,0.030076,0.006376,06-23-2023
5,23T1A 4,Vollebekk,112,5.0,NaN,Caress,SWKronjet//Tahti/SW38074,0,57,15,26,21,3.63,3098,11.7,24.640937,429.103373,NaN,0.841857,0.852093,0.045147,0.021933,0.021580,0.004473,0.037111,0.036428,0.008680,0.293318,0.299415,0.075772,0.141470,0.143143,0.034823,0.023932,0.023353,0.005169,06-23-2023
6,23T1A 4,Vollebekk,113,3.0,NaN,Betong,SW44254/Zebra//GN03531,0,64,15,25,23,3.96,3409,11.7,23.986187,472.179922,NaN,0.826331,0.833286,0.038390,0.024545,0.024283,0.004542,0.042251,0.041611,0.008843,0.297431,0.298764,0.063061,0.152854,0.152669,0.031785,0.027539,0.027074,0.005451,06-23-2023
7,23T1A 4,Vollebekk,114,20.0,NaN,SEC551-13-2,NaN,0,64,10,25,22,3.82,3301,11.7,23.69678,457.220863,NaN,0.849355,0.854038,0.031027,0.023197,0.022993,0.004178,0.041924,0.041496,0.008557,0.320302,0.322245,0.063854,0.160534,0.160930,0.031445,0.025522,0.025208,0.004929,06-23-2023
8,23T1A 4,Vollebekk,115,12.0,NaN,GN18636,GN07574/GN05567,0,72,10,27,22,4.56,4008,11.7,22.388947,555.147294,NaN,0.797549,0.811419,0.060420,0.023302,0.022861,0.004887,0.037727,0.036940,0.008869,0.256496,0.251709,0.069572,0.130050,0.127334,0.033130,0.027442,0.026809,0.006183,06-23-2023
9,23T1A 4,Vollebekk,116,19.0,NaN,GN19549,GN03509/SW51127,0,67,15,26,22,4.67,4023,11.7,23.933426,557.224941,NaN,0.814109,0.824850,0.048163,0.022941,0.022500,0.004893,0.037116,0.036315,0.008721,0.267369,0.266003,0.071264,0.132590,0.131758,0.034091,0.026136,0.025631,0.005310,06-23-2023


In [43]:
display(HTML(pd.concat([full_agro, phenocrop_2023_P4M_20m_20230623], axis=1).to_html()))


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [44]:
df1 = full_agro.iloc[:5,:5]

In [45]:
df2 = phenocrop_2023_P4M_20m_20230623.iloc[:10,:5]

In [46]:
df1

,r_expt,r_location,id,entry/bloc,
0,23T1A 4,Vollebekk,107,8.0,NaN
1,23T1A 4,Vollebekk,108,24.0,NaN
2,23T1A 4,Vollebekk,109,18.0,NaN
3,23T1A 4,Vollebekk,110,13.0,NaN
4,23T1A 4,Vollebekk,111,1.0,NaN


In [57]:
df2 = df2.drop(8)
df2

,id,ndvi_mean,ndvi_median,ndvi_stdev,blue_mean
0,101,0.756741,0.775178,0.075047,0.026580
1,102,0.772892,0.797868,0.090913,0.022288
2,103,0.774573,0.797225,0.078064,0.027562
3,104,0.795197,0.812887,0.067929,0.023070
4,105,0.795958,0.808139,0.054455,0.025474
5,106,0.803024,0.815190,0.054994,0.025777
6,107,0.744566,0.760509,0.072517,0.027839
7,108,0.813557,0.829784,0.060461,0.023538
9,110,0.814132,0.822713,0.045319,0.025654


In [58]:
pd.merge(df1, df2, on="id") 

,r_expt,r_location,id,entry/bloc,,ndvi_mean,ndvi_median,ndvi_stdev,blue_mean
0,23T1A 4,Vollebekk,107,8.0,NaN,0.744566,0.760509,0.072517,0.027839
1,23T1A 4,Vollebekk,108,24.0,NaN,0.813557,0.829784,0.060461,0.023538
2,23T1A 4,Vollebekk,110,13.0,NaN,0.814132,0.822713,0.045319,0.025654


In [50]:
display(HTML(pd.concat([df1, df2], axis=0).to_html()))


,r_expt,r_location,id,entry/bloc,,ndvi_mean,ndvi_median,ndvi_stdev,blue_mean
0,23T1A 4,Vollebekk,107,8.0,NaN,NaN,NaN,NaN,NaN
1,23T1A 4,Vollebekk,108,24.0,NaN,NaN,NaN,NaN,NaN
2,23T1A 4,Vollebekk,109,18.0,NaN,NaN,NaN,NaN,NaN
3,23T1A 4,Vollebekk,110,13.0,NaN,NaN,NaN,NaN,NaN
4,23T1A 4,Vollebekk,111,1.0,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,101,NaN,NaN,0.756741,0.775178,0.075047,0.026580
1,NaN,NaN,102,NaN,NaN,0.772892,0.797868,0.090913,0.022288
2,NaN,NaN,103,NaN,NaN,0.774573,0.797225,0.078064,0.027562
3,NaN,NaN,104,NaN,NaN,0.795197,0.812887,0.067929,0.023070
4,NaN,NaN,105,NaN,NaN,0.795958,0.808139,0.054455,0.025474


In [52]:
pd.merge(df1, df2, on="id") 

,r_expt,r_location,id,entry/bloc,,ndvi_mean,ndvi_median,ndvi_stdev,blue_mean
0,23T1A 4,Vollebekk,107,8.0,NaN,0.744566,0.760509,0.072517,0.027839
1,23T1A 4,Vollebekk,108,24.0,NaN,0.813557,0.829784,0.060461,0.023538
2,23T1A 4,Vollebekk,109,18.0,NaN,0.778521,0.791565,0.057855,0.025517
3,23T1A 4,Vollebekk,110,13.0,NaN,0.814132,0.822713,0.045319,0.025654
